## 无迹卡尔曼滤波
使用UKF进行移动机器人的位姿预测，见相关[博客](https://blog.csdn.net/weixin_42301220/article/details/124708187?csdn_share_tail=%7B%22type%22%3A%22blog%22%2C%22rType%22%3A%22article%22%2C%22rId%22%3A%22124708187%22%2C%22source%22%3A%22weixin_42301220%22%7D&ctrtid=B4W0n)


In [33]:
import PyQt5

In [34]:
import math
import matplotlib
import matplotlib.pyplot as plt
# %matplotlib inline
%matplotlib qt5

# # set up matplotlib
# is_ipython = 'inline' in matplotlib.get_backend()
# if is_ipython:
#     from IPython import display

plt.ion()

import scipy
import numpy as np
from scipy.spatial.transform import Rotation as Rot

设置过程噪声和观测噪声的协方差矩阵

In [35]:
Q = np.matrix([[0.1, 0, 0, 0],  # variance of location on x-axis
        [0, 0.1, 0, 0],  # variance of location on y-axis
        [0, 0, np.deg2rad(1.0), 0],  # variance of yaw angle
        [0, 0, 0, 1.0]])**2  # variance of velocity

R = np.matrix([[1.0,0.0],[0.0,1.0]])**2

仿真参数

In [36]:
#  Simulation parameter
INPUT_NOISE = np.diag([1.0, np.deg2rad(30.0)]) ** 2
GPS_NOISE = np.diag([0.5, 0.5]) ** 2

DT = 0.1  # time tick [s]
SIM_TIME = 50.0  # simulation time [s]

show_animation = True



UKF参数

使用比例无迹变换

![在这里插入图片描述](https://img-blog.csdnimg.cn/7df0f05a8c624ef497ccca3e89711562.png)

In [37]:
#  UKF Parameter
ALPHA = 0.001
BETA = 2
KAPPA = 0


UKF初始化：参数选择与权重计算
![在这里插入图片描述](https://img-blog.csdnimg.cn/ad4e2a00248b484e91804d018b517e8c.png)

In [52]:
def setup_ukf(nx):
    lamb = ALPHA ** 2 * (nx + KAPPA) - nx
    # calculate weights
    wm = [lamb / (lamb + nx)]
    wc = [(lamb / (lamb + nx)) + (1 - ALPHA ** 2 + BETA)]
    for i in range(2 * nx):
        wm.append(1.0 / (2 * (nx + lamb)))
        wc.append(1.0 / (2 * (nx + lamb)))
    gamma = math.sqrt(nx + lamb)

    wm = np.array([wm])
    wc = np.array([wc])

    return wm, wc, gamma


状态向量

x_t = [x,y,yaw,v]

控制输入为

u = [v,w]

控制输入表示

In [38]:
def control_input():
    v = 1.0  # 线速度，m/s
    yaw_rate = 0.1  #角速度，rad/s
    u = np.matrix([
            [v],
            [yaw_rate]
            ])
    return u

观测模型

In [39]:
def observation_model(x):
    H = np.matrix([[1.0,0,0,0],[0,1.0,0,0]])
    z = H@x
    return z


过程模型（状态方程）

In [40]:
def motion_model(x,u):
    F = np.matrix([
            [1.0,0,0,0],
            [0,1.0,0,0],
            [0,0,1.0,0],
            [0,0,0,0]
            ])
    B = np.matrix([
            [DT*np.cos(x[2,0]),0],
            [DT*np.sin(x[2,0]),0],
            [0,DT],
            [1.0,0]
            ])
    x = F@x+B@u
    return x
    

观测向量获取

In [41]:
def observation(x_true, x_d,u):
    x_true = motion_model(x_true,u)
    # add noise to gps x-y
    z = observation_model(x_true) + GPS_NOISE @ np.random.randn(2, 1)

    # add noise to input
    ud = u + INPUT_NOISE @ np.random.randn(2, 1)

    xd = motion_model(x_d, ud)

    return x_true, z, xd, ud


sigma点采样

![在这里插入图片描述](https://img-blog.csdnimg.cn/6c6d31779f674b8aa27c0d2b5a351065.png)

In [42]:
def generate_sigma_points(xEst, PEst, gamma):
    sigma = xEst
    Psqrt = scipy.linalg.sqrtm(PEst)
    n = len(xEst[:, 0])
    # Positive direction
    for i in range(n):
        sigma = np.hstack((sigma, xEst + gamma * Psqrt[:, i:i + 1]))

    # Negative direction
    for i in range(n):
        sigma = np.hstack((sigma, xEst - gamma * Psqrt[:, i:i + 1]))

    return sigma


状态转移非线性变换

![在这里插入图片描述](https://img-blog.csdnimg.cn/28c6485d04ee432f849ed12d95b8f455.png)

In [43]:
def predict_sigma_motion(sigma, u):
    """
        Sigma Points prediction with motion model
    """
    for i in range(sigma.shape[1]):
        sigma[:, i:i + 1] = motion_model(sigma[:, i:i + 1], u)

    return sigma

观测非线性变换
![在这里插入图片描述](https://img-blog.csdnimg.cn/3e97c759dadf4915a8725e26723e91ac.png)

In [44]:
def predict_sigma_observation(sigma):
    """
        Sigma Points prediction with observation model
    """
    for i in range(sigma.shape[1]):
        sigma[0:2, i] = observation_model(sigma[:, i])

    sigma = sigma[0:2, :]

    return sigma


 加权计算 k 时刻状态量的先验概率分布
 ![在这里插入图片描述](https://img-blog.csdnimg.cn/5112c5661d54476ab1ca93b585ff8c30.png)

In [45]:
def calc_sigma_covariance(x, sigma, wc, Pi):
    nSigma = sigma.shape[1]
    d = sigma - x[0:sigma.shape[0]]
    P = Pi
    for i in range(nSigma):
        P = P + wc[0, i] * d[:, i:i + 1] @ d[:, i:i + 1].T
    return P


计算状态量与观测量的互协方差

![在这里插入图片描述](https://img-blog.csdnimg.cn/a9aa0a94ca69404ba47cf2c90066ec4a.png)

In [46]:
def calc_pxz(sigma, x, z_sigma, zb, wc):
    nSigma = sigma.shape[1]
    dx = sigma - x
    dz = z_sigma - zb[0:2]
    P = np.zeros((dx.shape[0], dz.shape[0]))

    for i in range(nSigma):
        P = P + wc[0, i] * dx[:, i:i + 1] @ dz[:, i:i + 1].T

    return P


UKF估计

代码符号说明
- 第1个sigma：![在这里插入图片描述](https://img-blog.csdnimg.cn/de69dc49635c4139bbda6883588d72ba.png)

- 第2个sigma：![在这里插入图片描述](https://img-blog.csdnimg.cn/7bdb084770ee43758f98654b61895bcc.png)

- xPred：![在这里插入图片描述](https://img-blog.csdnimg.cn/330e6716828548c78cc39bbb7f5803fc.png)

- PPred：![在这里插入图片描述](https://img-blog.csdnimg.cn/cf3a523bb046484abbe9d110e9c7b3ca.png)

- 第3个sigma：![在这里插入图片描述](https://img-blog.csdnimg.cn/8cd1bcd7ea5449f5b62ccd85a3f29917.png)

- z_sigma：![在这里插入图片描述](https://img-blog.csdnimg.cn/3af2cd6f1fb74c84a064899fc760fa05.png)

- zb：![在这里插入图片描述](https://img-blog.csdnimg.cn/d9a560bb9e58448598f9f6f431abbe34.png)

- st：![在这里插入图片描述](https://img-blog.csdnimg.cn/8c8d1b74d1c94007b778cec6bf2d4cef.png)

- Pxz：![在这里插入图片描述](https://img-blog.csdnimg.cn/e22ba23b218b48f8aa2bfe7ca4d92e22.png)

- K：卡尔曼增益![在这里插入图片描述](https://img-blog.csdnimg.cn/5315544217454febac7535f777823ee5.png)

- z：![在这里插入图片描述](https://img-blog.csdnimg.cn/ebe9e42b0487460fa36f031bca28077a.png)

- y：![在这里插入图片描述](https://img-blog.csdnimg.cn/053c058490874d4fae78be4eb82cbfdc.png)



In [54]:
def ukf_estimation(xEst, PEst, z, u, wm, wc, gamma):
    #  Predict
    
    ## 1.sigma采样
    sigma = generate_sigma_points(xEst, PEst, gamma)
    
    ## 2.状态转移非线性变换
    sigma = predict_sigma_motion(sigma, u)
    
    ## 3.加权计算 k 时刻状态量的先验概率分布
    xPred = (wm @ sigma.T).T
    PPred = calc_sigma_covariance(xPred, sigma, wc, Q)

    #  Update

    ## 4.对 k 时刻状态量的先验概率分布进行 sigma 采样
    sigma = generate_sigma_points(xPred, PPred, gamma)
    

    
    ## 5.观测非线性变换
    z_sigma = predict_sigma_observation(sigma)
    
    ## 6.加权计算k时刻观测量 Z_{k}的概率分布
    # zb = (wm @ sigma.T).T
    zb = (wm @ z_sigma.T).T
    st = calc_sigma_covariance(zb, z_sigma, wc, R)
    
    ## 7. 计算状态量与观测量的互协方差
    Pxz = calc_pxz(sigma, xPred, z_sigma, zb, wc)
    
    ## 8. 计算卡尔曼增益
    K = Pxz @ np.linalg.inv(st)
    
    ## 计算 k 时刻状态量的后验概率分布
    zPred = observation_model(xPred)
    # y = z - zPred
    y = z - zb
    xEst = xPred + K @ y
    PEst = PPred - K @ st @ K.T

    return xEst, PEst


画图

In [55]:
def plot_covariance_ellipse(xEst, PEst):  # pragma: no cover
    """The blue line is true trajectory, the black line is dead reckoning trajectory,
    the green point is positioning observation (ex. GPS), and the red line is estimated trajectory with EKF.

    The red ellipse is estimated covariance ellipse with UKF.

    Args:
        xEst (_type_): _description_
        PEst (_type_): _description_
    """
    Pxy = PEst[0:2, 0:2]
    eigval, eigvec = np.linalg.eig(Pxy)

    if eigval[0] >= eigval[1]:
        bigind = 0
        smallind = 1
    else:
        bigind = 1
        smallind = 0

    t = np.arange(0, 2 * math.pi + 0.1, 0.1)
    a = math.sqrt(eigval[bigind])
    b = math.sqrt(eigval[smallind])
    x = [a * math.cos(it) for it in t]
    y = [b * math.sin(it) for it in t]
    angle = math.atan2(eigvec[1, bigind], eigvec[0, bigind])
    rot = Rot.from_euler('z', angle).as_matrix()[0:2, 0:2]
    fx = rot @ np.array([x, y])
    px = np.array(fx[0, :] + xEst[0, 0]).flatten()
    py = np.array(fx[1, :] + xEst[1, 0]).flatten()
    plt.plot(px, py, "--r")

主函数入口

In [53]:
def main():
    nx = 4  # State Vector [x y yaw v]'
    xEst = np.zeros((nx, 1))
    xTrue = np.zeros((nx, 1))
    PEst = np.eye(nx)
    xDR = np.zeros((nx, 1))  # Dead reckoning

    wm, wc, gamma = setup_ukf(nx)

    # history
    hxEst = xEst
    hxTrue = xTrue
    hxDR = xTrue
    hz = np.zeros((2, 1))

    time = 0.0

    while SIM_TIME >= time:
        time += DT
        u = control_input()

        xTrue, z, xDR, ud = observation(xTrue, xDR, u)

        xEst, PEst = ukf_estimation(xEst, PEst, z, ud, wm, wc, gamma)

        # store data history
        hxEst = np.hstack((hxEst, xEst))
        hxDR = np.hstack((hxDR, xDR))
        hxTrue = np.hstack((hxTrue, xTrue))
        hz = np.hstack((hz, z))

        if show_animation:
                plt.cla()
                # for stopping simulation with the esc key.
                plt.gcf().canvas.mpl_connect('key_release_event',lambda event: [exit(0) if event.key == 'escape' else None])
                plt.plot(hz[0, :], hz[1, :], ".g")
                plt.plot(np.array(hxTrue[0, :]).flatten(),
                        np.array(hxTrue[1, :]).flatten(), "-b")
                plt.plot(np.array(hxDR[0, :]).flatten(),
                        np.array(hxDR[1, :]).flatten(), "-k")
                plt.plot(np.array(hxEst[0, :]).flatten(),
                        np.array(hxEst[1, :]).flatten(), "-r")
                plot_covariance_ellipse(xEst, PEst)
                plt.axis("equal")
                plt.grid(True)
                plt.pause(0.001)
        plt.savefig('./result.png')


In [56]:
main()